# Indexing and Selecting data


---

## Learning Objectives 


- Select data by position using `.isel()` with values or slices
- Select data by coordinate label/value using `.sel()` with values or slices
- Use nearest-neighbor lookups with `.sel()`
- Use `interp()` to interpolate by coordinate labels

## Prerequisites


| Concepts | Importance | Notes |
| --- | --- | --- |
| [Understanding of xarray core data structures](./01-xarray-fundamentals.ipynb) | Necessary | |
| [Basic familiarity with NumPy indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html) | Helpful | |
| [Basic familiarity with Pandas indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) | Helpful | |

- **Time to learn**: *15-20 minutes*



---

## Imports


In [3]:
import xarray as xr

In [4]:
ds = xr.open_dataset(
    "/home/rdevinen/palm/current_version/JOBS/testing/INPUT/testing_static", engine="netcdf4")

ds

<xarray.Dataset>
Dimensions:            (x: 64, y: 64, z: 65, zlad: 6, zsoil: 8, nsurface_fraction: 3)
Coordinates:
  * x                  (x) float32 0.5 1.5 2.5 3.5 4.5 ... 60.5 61.5 62.5 63.5
  * y                  (y) float32 0.5 1.5 2.5 3.5 4.5 ... 60.5 61.5 62.5 63.5
  * z                  (z) float32 0.0 0.5 1.5 2.5 3.5 ... 60.5 61.5 62.5 63.5
  * zlad               (zlad) float32 0.0 0.5 1.5 2.5 3.5 4.5
  * zsoil              (zsoil) float32 0.005 0.02 0.05 0.1 0.2 0.4 0.8 2.0
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Data variables:
    building_id        (y, x) float64 ...
    buildings_2d       (y, x) float32 ...
    buildings_3d       (z, y, x) float32 ...
    zt                 (y, x) float32 ...
    building_type      (y, x) float32 ...
    pavement_type      (y, x) float32 ...
    soil_type          (y, x) float32 ...
    street_type        (y, x) float32 ...
    surface_fraction   (nsurface_fraction, y, x) float32 ...
    vegetation_type    (y, x) float32 ...
    water_type         (y, x) float32 ...
    lad                (zlad, y, x) float32 ...
Attributes: (12/20)
    title:           Example PALM static driver
    author:          PALM user
    institution:     Institut of Meteorology and Climatology,Leibniz Universi...
    comment:         Generic crossing example
    creation_date:   22-03-22 11:02:43 +00
    history:         
    ...              ...
    origin_lon:      7.944378
    origin_time:     2019-03-06 10:00:00 +00
    origin_x:        3422021.0
    origin_y:        5370972.0
    origin_z:        0.0
    rotation_angle:  0.0

## NumPy Positional Indexing

When working with numpy, indexing is done by position (slices/ranges/scalars).

In [6]:
sf = ds["surface_fraction"].data  # retrieve numpy array
sf

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)

In [7]:
sf.shape, sf.ndim

((3, 64, 64), 3)

Let's extract a timeseries for a single spatical location 


In [10]:
sf[:, 32, 0]

array([0., 0., 0.], dtype=float32)

## Different choices for indexing 


Xarray supports two kinds of indexing 

- Positional indexing via `.isel()`: provides primarily integer position based index (from `0` to `length-1` of the axis/dimension
- Label indexing via `.sel()`: provides primarily label based index

Xarray's indexing methods preserves the coordinate labels and associated metadata.



### Selection by position

The `.isel()` method is the primary access method for **purely integer based indexing**. The following are valid inputs:
- An integer e.g. `lat=10`
- A list or array of integers `lon=[10, 20, 39]`
- A slice object with integers e.g. `time=slice(2, 20)`

In [14]:
ds["surface_fraction"].isel()  # the original object i.e. no selection

<xarray.DataArray 'surface_fraction' (nsurface_fraction: 3, y: 64, x: 64)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * x                  (x) float32 0.5 1.5 2.5 3.5 4.5 ... 60.5 61.5 62.5 63.5
  * y                  (y) float32 0.5 1.5 2.5 3.5 4.5 ... 60.5 61.5 62.5 63.5
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Attributes:
    long_name:  surface fraction
    units:      1

In [15]:
ds["surface_fraction"].isel(x=32)

<xarray.DataArray 'surface_fraction' (nsurface_fraction: 3, y: 64)>
array([[0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)
Coordinates:
    x                  float32 32.5
  * y                  (y) float32 0.5 1.5 2.5 3.5 4.5 ... 60.5 61.5 62.5 63.5
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Attributes:
    long_name:  surface fraction
    units:      1

In [16]:
ds["surface_fraction"].isel(x=32, y=0)

<xarray.DataArray 'surface_fraction' (nsurface_fraction: 3)>
array([0., 0., 0.], dtype=float32)
Coordinates:
    x                  float32 32.5
    y                  float32 0.5
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Attributes:
    long_name:  surface fraction
    units:      1

In [17]:
ds["surface_fraction"].isel(x=32, y=slice(10, 35))

<xarray.DataArray 'surface_fraction' (nsurface_fraction: 3, y: 25)>
array([[1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)
Coordinates:
    x                  float32 32.5
  * y                  (y) float32 10.5 11.5 12.5 13.5 ... 31.5 32.5 33.5 34.5
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Attributes:
    long_name:  surface fraction
    units:      1

### Selection by label 


The `.sel()` method is the primary access method for **purely coordinate label based indexing.**. The following are valid inputs:

- A single coordinate label e.g. `time="2021-03-01"`
- A list or array of coordinate labels `lon=[="2021-01-01", ="2021-03-10", ="2021-03-12"]`
- A slice object with coordinate labels e.g. `time=slice("2021-01-01", "2021-03-01")`.  (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index!)

In [20]:
ds["surface_fraction"].sel(y=12.5, method='nearest')

<xarray.DataArray 'surface_fraction' (nsurface_fraction: 3, x: 64)>
array([[0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)
Coordinates:
  * x                  (x) float32 0.5 1.5 2.5 3.5 4.5 ... 60.5 61.5 62.5 63.5
    y                  float32 12.5
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Attributes:
    long_name:  surface fraction
    units:      1

### Nearest-neighbor lookups

As shown above, when our coordinate labels are not integers or strings or datetime-like but floating point numbers, `.sel()` may throw a `KeyError`:

`ds.tas.sel(lat=39.5, lon=105.7)` fails because we are trying to use a conditional for an approximate value i.e floating numbers are represented approximately inside the computer, and xarray is unable to locate this exact value. To address this issue, xarray supports `method` and `tolerance` keyword argument. The `method` parameter allows for enabling nearest neighbor (inexact) lookups by use of the methods `'pad', 'backfill' or 'nearest'`: 

In [21]:
ds["surface_fraction"].sel(x=32, y=0, method='nearest')

<xarray.DataArray 'surface_fraction' (nsurface_fraction: 3)>
array([0., 0., 0.], dtype=float32)
Coordinates:
    x                  float32 32.5
    y                  float32 0.5
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Attributes:
    long_name:  surface fraction
    units:      1

See the [xarray documentation](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.sel.html) for more on usage of `method` and `tolerance` parameters in `.sel()`. 

<div class="admonition alert alert-info">
    <p class="title" style="font-weight:bold">Tip</p>
Another way to use the nearest neighbor lookup is via slice objects. For e.g.:
</div>

In [23]:
ds["surface_fraction"].sel(x=slice(31, 33), y=slice(0, 1))

<xarray.DataArray 'surface_fraction' (nsurface_fraction: 3, y: 1, x: 2)>
array([[[0., 0.]],

       [[0., 0.]],

       [[0., 0.]]], dtype=float32)
Coordinates:
  * x                  (x) float32 31.5 32.5
  * y                  (y) float32 0.5
  * nsurface_fraction  (nsurface_fraction) int32 0 1 2
Attributes:
    long_name:  surface fraction
    units:      1

Operators can be chained, so multiple operations can be peformed sequentially. For example, to select an area of interest and the first time index

In [25]:
ds["surface_fraction"].isel(nsurface_fraction=0).sel(x=slice(31, 33), y=slice(0, 1))

<xarray.DataArray 'surface_fraction' (y: 1, x: 2)>
array([[0., 0.]], dtype=float32)
Coordinates:
  * x                  (x) float32 31.5 32.5
  * y                  (y) float32 0.5
    nsurface_fraction  int32 0
Attributes:
    long_name:  surface fraction
    units:      1

### Interpolation

If we want to interpolate along coordinates rather than looking up the nearest neighbos, we can use the `.interp()` method. To use `interp()` requires the presence of `scipy` library. 


In [21]:
ds["surface_fraction"].interp(lat=[10, 10.1, 10.2], method='nearest')

<xarray.DataArray 'tas' (time: 180, lat: 3, lon: 288)>
array([[[299.10678, 298.3701 , 297.9283 , ..., 297.73807, 298.29514,
         299.2525 ],
        [299.10678, 298.3701 , 297.9283 , ..., 297.73807, 298.29514,
         299.2525 ],
        [299.10678, 298.3701 , 297.9283 , ..., 297.73807, 298.29514,
         299.2525 ]],

       [[300.10565, 298.89124, 298.5006 , ..., 298.87277, 299.31076,
         300.50803],
        [300.10565, 298.89124, 298.5006 , ..., 298.87277, 299.31076,
         300.50803],
        [300.10565, 298.89124, 298.5006 , ..., 298.87277, 299.31076,
         300.50803]],

       [[304.44867, 303.03387, 302.48474, ..., 303.73337, 303.93005,
         305.0411 ],
        [304.44867, 303.03387, 302.48474, ..., 303.73337, 303.93005,
         305.0411 ],
        [304.44867, 303.03387, 302.48474, ..., 303.73337, 303.93005,
         305.0411 ]],
...
       [[303.61374, 301.92783, 300.61765, ..., 301.1961 , 302.29883,
         303.4983 ],
        [303.61374, 301.92783, 300.61765, ..., 301.1961 , 302.29883,
         303.4983 ],
        [303.61374, 301.92783, 300.61765, ..., 301.1961 , 302.29883,
         303.4983 ]],

       [[303.9199 , 302.7812 , 300.5473 , ..., 302.38495, 303.12256,
         304.22882],
        [303.9199 , 302.7812 , 300.5473 , ..., 302.38495, 303.12256,
         304.22882],
        [303.9199 , 302.7812 , 300.5473 , ..., 302.38495, 303.12256,
         304.22882]],

       [[301.18817, 300.2263 , 299.42358, ..., 299.85117, 300.60828,
         301.4995 ],
        [301.18817, 300.2263 , 299.42358, ..., 299.85117, 300.60828,
         301.4995 ],
        [301.18817, 300.2263 , 299.42358, ..., 299.85117, 300.60828,
         301.4995 ]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 2000-01-15 12:00:00 ... 2014-12-15 12:00:00
  * lat      (lat) float64 10.0 10.1 10.2
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        TREFHT
    description:    near-surface (usually, 2 meter) air temperature
    frequency:      mon
    id:             tas
    long_name:      Near-Surface Air Temperature
    mipTable:       Amon
    out_name:       tas
    prov:           Amon ((isd.003))
    realm:          atmos
    standard_name:  air_temperature
    time:           time
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Near-Surface Air Temperature
    type:           real
    units:          K
    variable_id:    tas

---

## Summary 

- Xarray’s named dimensions and labeled coordinates free the user from having to track positional ordering of dimensions when accessing data
- Xarray provides a variety of methods for subsetting data via `.sel()`, `.isel()`, `.interp()` methods


## Resources and References

- [Xarray Documentation - Indexing and Selecting Data](https://xarray.pydata.org/en/stable/indexing.html)
- [Xarray Documentation - Interpolation](https://xarray.pydata.org/en/stable/user-guide/interpolation.html)


<div class="admonition alert alert-success">
    <p class="title" style="font-weight:bold">Previous: <a href="./01-xarray-fundamentals.ipynb">Xarray Fundamentals</a></p>
    <p class="title" style="font-weight:bold">Next: <a href="./03-data-visualization.ipynb">Data Visualization</a></p>
</div>